<a href="https://colab.research.google.com/github/Shriansh16/Advanced_GenAi/blob/main/20_implementing_hybrid_search.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install --upgrade --quiet  pinecone-client pinecone-text pinecone-notebooks

  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 244.8/244.8 kB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.6/67.6 kB 5.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.4/85.4 kB 6.9 MB/s eta 0:00:00


In [2]:
api_key=""

In [5]:
from langchain_community.retrievers import PineconeHybridSearchRetriever

In [6]:
import os
from pinecone import Pinecone,ServerlessSpec
index_name="hybrid-search-test16"
## initialize the Pinecone client
pc=Pinecone(api_key=api_key)

#create the index
if index_name not in pc.list_indexes().names():
    pc.create_index(
        name=index_name,
        dimension=384,  # dimensionality of dense model
        metric="dotproduct",  # sparse values supported only for dotproduct
        spec=ServerlessSpec(cloud="aws", region="us-east-1"),
    )

In [13]:
index=pc.Index(index_name)
index

In [9]:
import os
## vector embedding and sparse matrix

os.environ["HF_TOKEN"]="pcsk_qSnAY_LArwtkk3zg6iQDk4A15GNHApK2FsKcSyKq6RYm5TjsoqZEDbdgKnkSuqYw4A51k"
from langchain.embeddings import HuggingFaceEmbeddings
embedding = HuggingFaceEmbeddings(model_name='sentence-transformers/all-MiniLM-L6-v2')

<ipython-input-9-a980099a28bd>:6: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embedding = HuggingFaceEmbeddings(model_name='sentence-transformers/all-MiniLM-L6-v2')
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or dat

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.7k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [10]:
from pinecone_text.sparse import BM25Encoder

bm25_encoder=BM25Encoder()
bm25_encoder

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [18]:
sentences=[
        "Liam lived in a village surrounded by mountains and forests. One day, he found a map.",
        "The map revealed a treasure hidden deep within the Enchanted Forest nearby.",
        "He packed his bag and set out with Max, his loyal dog, on an adventure.",
        "They crossed a wide river with strong currents, testing their courage and strength.",
        "While resting by a fire, they were visited by an owl with a mysterious message.",
        "The owl's clue led them to a cave hidden beneath a massive waterfall.",
        "Inside the cave, they found a locked door with a riddle carved on it.",
        "After solving the riddle, they discovered a glowing book inside the chamber.",
        "The book granted them wisdom and the key to uncover even greater secrets.",
        "Liam returned to the village as a hero, his journey inspiring everyone there."

]

## tfidf values on these sentence
bm25_encoder.fit(sentences)

  0%|          | 0/10 [00:00<?, ?it/s]

In [17]:
import nltk
nltk.download('punkt_tab')

[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.


True

In [14]:
retriever=PineconeHybridSearchRetriever(embeddings=embedding,sparse_encoder=bm25_encoder,index=index)

In [19]:
retriever.add_texts(
    [
        "Liam lived in a village surrounded by mountains and forests. One day, he found a map.",
        "The map revealed a treasure hidden deep within the Enchanted Forest nearby.",
        "He packed his bag and set out with Max, his loyal dog, on an adventure.",
        "They crossed a wide river with strong currents, testing their courage and strength.",
        "While resting by a fire, they were visited by an owl with a mysterious message.",
        "The owl's clue led them to a cave hidden beneath a massive waterfall.",
        "Inside the cave, they found a locked door with a riddle carved on it.",
        "After solving the riddle, they discovered a glowing book inside the chamber.",
        "The book granted them wisdom and the key to uncover even greater secrets.",
        "Liam returned to the village as a hero, his journey inspiring everyone there."
    ]
)

  0%|          | 0/1 [00:00<?, ?it/s]

In [20]:
retriever.invoke("What did Liam find that started his adventure?")

[Document(metadata={'score': 0.371556163}, page_content='Liam lived in a village surrounded by mountains and forests. One day, he found a map.'),
 Document(metadata={'score': 0.337031692}, page_content='Liam returned to the village as a hero, his journey inspiring everyone there.'),
 Document(metadata={'score': 0.27078855}, page_content='He packed his bag and set out with Max, his loyal dog, on an adventure.'),
 Document(metadata={'score': 0.184407949}, page_content='The map revealed a treasure hidden deep within the Enchanted Forest nearby.')]

In [21]:
retriever.invoke("What did Liam bring along on his journey?")

[Document(metadata={'score': 0.453010798}, page_content='Liam returned to the village as a hero, his journey inspiring everyone there.'),
 Document(metadata={'score': 0.348395973}, page_content='Liam lived in a village surrounded by mountains and forests. One day, he found a map.'),
 Document(metadata={'score': 0.162097663}, page_content='He packed his bag and set out with Max, his loyal dog, on an adventure.'),
 Document(metadata={'score': 0.125142992}, page_content='They crossed a wide river with strong currents, testing their courage and strength.')]